In [1]:
import numpy as np
import pandas as pd
import keras
import cv2
import matplotlib.pyplot as plt
from spp.SpatialPyramidPooling import SpatialPyramidPooling
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input, Flatten
from keras.models import Model
from keras.activations import relu
import keras
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
data = pd.read_csv("./training-data/train.csv", names = ['image','label'])

In [3]:
data = data.sample(n=500, random_state=14)

In [4]:
print(len(np.unique(data.label)), data.shape)

35 (500, 2)


In [ ]:
classes = len(np.unique(data.label))

In [ ]:
data_labels = keras.utils.to_categorical(data.label, num_classes=43)
trainX, testX, trainY, testY = train_test_split(data, data_labels, test_size = 0.1, random_state = 42)
trainX = trainX.reset_index().drop('index', axis=1)
testX = testX.reset_index().drop('index', axis=1)

In [ ]:
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

(450, 2) (50, 2) (450, 43) (50, 43)


In [ ]:
# data = []
# labels = []

# for i in tqdm(range(len(data))):
#     labels

### Data Generator

In [ ]:
def data_gen(gen_data, batch_size):
    idx_start = 0
    if gen_data == 'train':
        data = trainX
        data_labels = trainY
    elif gen_data == 'test':
        data = testX
        data_labels = testY
    
    label_batch = np.zeros((batch_size, 43))
    
    while True:
        image_batch =[]
        batch_i = 0
        for idx in range(idx_start, idx_start + batch_size):
            if idx < data.shape[0]:
                #print(idx)
                file = './training-data/train-images/' + data.image[idx]
                image = cv2.imread(file)
                image = cv2.resize(image, (224,224))
                image_batch.append(image)
                label_batch[batch_i] = data_labels[idx]
            
            else:
                idx_start = 1
            batch_i += 1
        idx_start += batch_size
        image_batch =np.array(image_batch) /255.0
#         print(image_batch.shape, label_batch.shape)
        yield image_batch, label_batch

In [ ]:
input_shape = (224,224,3)
epochs = 2
batch_size = 10

In [ ]:
base_model = keras.applications.mobilenet_v2.MobileNetV2(input_shape=input_shape, include_top=True, weights='imagenet')

In [ ]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [ ]:
out = base_model.get_layer('global_average_pooling2d_1').output
# out = Flatten(data_format=)(out)
out = Dense(43, activation='relu')(out)
model = Model(input= base_model.input, output= out)
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

/home/godara01/.virtualenv/ml/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  after removing the cwd from sys.path.


In [ ]:
#callbacks
early_stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None)
checkpointer = keras.callbacks.ModelCheckpoint(filepath='./model.hdf5', verbose=1, save_best_only=True)

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):

    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
        val_targ = self.model.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(" — val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
        return

conf_metrics = Metrics()

In [ ]:
history = model.fit_generator(data_gen('train', batch_size),
                              callbacks=[early_stopper, checkpointer],
                              validation_data= data_gen('test', batch_size),
                              validation_steps=testX.shape[0]//batch_size,
                              epochs=epochs, steps_per_epoch = trainX.shape[0]//batch_size)

Epoch 1/2
35/45 [======================>.......] - ETA: 39s - loss: 9.0200 - acc: 0.0886

In [ ]:
model.evaluate_generator(data_gen('test', batch_size), steps=10)